In [1]:
import os
from getpass import getpass

os.environ["GOOGLE_API_KEY"] = getpass("Enter your Gemini API key: ")

In [ ]:
import os

os.environ['FIFTYONE_ALLOW_LEGACY_ORCHESTRATORS'] = 'true'

Download dataset from HF and take a small sample:

In [7]:
import fiftyone as fo
import fiftyone.utils.huggingface as fouh

dataset = fouh.load_from_hub("Voxel51/ScreenSpot")

smol_ds = dataset.take(3).clone(name="smol_spot")

Install plugins and download requirements:

In [ ]:
!fiftyone plugins download https://github.com/harpreetsahota204/agno-fo-test

In [ ]:
!fiftyone plugins requirements @harpreetsahota/agnoagent --install

You can use the operator in the app.

First, launch the app:

```python

import fiftyone as fo

fo.launch_app(smol_ds)

```

Then hit the backtick key on your keyboard (\`\), this will open the operator panel. Type in `Agno Agent` and fill out the form:

<img src="assets/agno_agent_app.gif">


You can also run the plugin via the sdk:

In [ ]:
import fiftyone.operators as foo

agno_agent = foo.get_operator("@harpreetsahota/agnoagent/agno_agent")

In [17]:
SYSTEM_MESSAGE = """Given an image of a GUI screenshot and a corresponding instruction, the task is to 
output one bounding box for the relevant GUI element in the screenshot that correspond to the instruction and 
associated with one of the following labels: text or icon. 
"""

Kick off a delegated service, open your terminal and type: `fiftyone delegated launch`.

Then run the following cell:

In [ ]:
await agno_agent(
    smol_ds,
    operation_mode="prompt_field",
    output_field="field_predictions", # you can name this whatever you want
    system_message=SYSTEM_MESSAGE,
    prompt_field="instruction", # this should be the field in the dataset that contains the instruction
    verbose=True,
)

Alternatively, you can pass in a a free form instruction which will be used on each Sample in the dataset:

In [ ]:
await agno_agent(
    smol_ds,
    operation_mode="text_prompt",
    output_field="predictions",      # where to store results
    system_message=SYSTEM_MESSAGE,
    text_prompt="Find all buttons in this image",  # prompt applied to all images
    verbose=True,                   # enable debugging output
)